In [ ]:
from pathlib import Path
import time

from tqdm import tqdm
import numpy as np
from scipy import ndimage
import nibabel as nib
import matplotlib.pyplot as plt

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [ ]:
from bcblib.tools import blob_robot
from nilearn.plotting import plot_glass_brain, plot_stat_map

template_hdr = nib.load('/home/tolhsadum/neuro_apps/sync_project/lesseg_unet/lesseg_unet/data/avg152T2.nii')
cell_array = blob_robot.create_cell_array((91, 109, 91), 0)
# cell_array[50:55, 65:72, 45:55] = blob_robot.Cell(4)
# cell_array = blob_robot.create_cell_array((10, 10, 10), 0)
# data = blob_robot.cell_array_to_state_array(cell_array)
# cell_array[1:4, 1:4, 1:4] = blob_robot.Cell(4)


# rule = (4, 5, 2, 'M')
rule = [[range(4, 6)], 3, 2, 'M']
# rule = [[3,5,7,9,11,15,17,19,21,23, 24,26],[3,6,8,9,11,14, 15, 16,17,19,24],7, 'M']
# Pyroclastic (Jason Rampe) 4-7/6-8/10/M
# rule = [[range(4, 7)], [range(6, 8)], 10, 'M']
# 0-3,7-9,11-13,18,21-22,24,26/13,17,20-26/4/M Spike growth
# rule = [[range(0, 3), range(7, 9), range(11, 13), 18, 21, 22, 24, 26], [13, 17, range(20, 26)], 4, 'M']
# Coral (Jason Rampe) 5-8/6-7,9,12/4/M
# rule = [[range(5, 8)], [6, 7, 9, 12], 4, 'M']
# rule = [1, 3, 5, 7, 11, 13, [range(2-4), range(8-10)], 3, 'M']


acc = 0
# cube_start, cube_end = 44, 47
# cube_start, cube_end = 4, 12
# for x in range(cube_start, cube_end):
#     for y in range(cube_start, cube_end):
#         for z in range(cube_start, cube_end):
#             if cube_start in (x, y, z) or cube_end in (x, y, z):
# #             if x % 3 and y % 3:
#                 c = cell_array[x, y, z]
#                 acc += 1
#                 c.set_next_state(rule[2] - 1, 0)
#                 c.update_state()
#
# for x in range(cell_array.shape[0]):
#     for z in range(cell_array.shape[2]):
#         c = cell_array[x, 0, z]
#         c.set_next_state(rule[2] - 1, 0)
#         c.update_state()
#         if x % 2:
#             c = cell_array[x, 1, z]
#             c.set_next_state(rule[2] - 1, 0)
#             c.update_state()
#         if z % 2:
#             c = cell_array[x, 1, z]
#             c.set_next_state(rule[2] - 1, 0)
#             c.update_state()

blob_robot.create_shapes_in_arr(cell_array, coords=8, connectivity=4)

data = blob_robot.cell_array_to_state_array(cell_array, value_mode='spawn')
print(f'Number of live cells: {data[np.where(data > 0)].size}')
print(f'Number of unique values: {np.unique(data)}')
new_img = nib.Nifti1Image(data + 1, template_hdr.affine, dtype=np.uint16)
centre_of_mass = ndimage.center_of_mass(data + 1)
display = plot_stat_map(new_img, output_file=None, cut_coords=centre_of_mass, cmap='hsv', colorbar=True, draw_cross=False)
plt.show()


for it in tqdm(range(50)):
    start = time.time()
    blob_robot.evolve_automaton(cell_array, it, rule=rule)

    end = time.time()
    # print(f'Iteration {it} time: {end - start}')
    # data = blob_robot.cell_array_to_state_array(cell_array, value_mode='spawn')
    # print(f'Number of live cells: {data[np.where(data > 0)].size}')
    # print(f'Number of unique values: {np.unique(data)}')
    # new_img = nib.Nifti1Image(data + 1, template_hdr.affine, dtype=np.uint16)
    # nib.save(new_img, '/home/tolhsadum/neuro_apps/data/test_blob_robot_try.nii.gz')
    # centre_of_mass = ndimage.center_of_mass(data + 1)
    # plot_stat_map(new_img, output_file=None, cut_coords=centre_of_mass, cmap='hsv', colorbar=True, draw_cross=False)
    # plt.show()

data = blob_robot.cell_array_to_state_array(cell_array, value_mode='spawn')
print(f'Number of live cells: {data[np.where(data > 0)].size}')
print(f'Number of unique values: {np.unique(data)}')
new_img = nib.Nifti1Image(data + 1, template_hdr.affine, dtype=np.uint16)
nib.save(new_img, '/home/tolhsadum/neuro_apps/data/test_blob_robot_try.nii.gz')
centre_of_mass = ndimage.center_of_mass(data + 1)
plot_stat_map(new_img, output_file=None, cut_coords=centre_of_mass, cmap='hsv', colorbar=True, draw_cross=False)
plt.show()